# experiments.ipynb

This notebook contains experiments using various methods and libraries to see what gets us the best running time and memory usage.

In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import sys
sys.path.append("../..")

import pipit as pp
import pandas as pd
import dask
import numpy as np
import sqlite3
import polars
from operator import itemgetter
import duckdb
import json

Let's read in the sw4lite-32 OTF2 dataset:

In [14]:
trace = pp.Trace.from_otf2("/home/rakrish/trace-data/sw4lite-32/")

# DataFrame (stored column-wise)
df = trace.events
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
0,4.772924e-01,Instant,ProgramBegin,0,14,{'program_name': '/g/g92/bhowmik1/sw4lite/sw4l...
1,1.484379e+02,Instant,ProgramBegin,0,15,{'program_name': '/g/g92/bhowmik1/sw4lite/sw4l...
2,1.740208e+03,Instant,ProgramBegin,0,12,{'program_name': '/g/g92/bhowmik1/sw4lite/sw4l...
3,2.002576e+04,Instant,ProgramBegin,0,13,{'program_name': '/g/g92/bhowmik1/sw4lite/sw4l...
4,2.014269e+04,Enter,MPI_Init,0,14,{'region': 'Region 36'}
...,...,...,...,...,...,...
9533,4.699147e+09,Leave,MPI_Sendrecv,0,25,None
9534,4.704243e+09,Leave,TRACER_Loop,0,25,None
9535,4.704245e+09,Instant,MeasurementOnOff,0,25,{'measurement_mode': 'MeasurementMode.OFF'}
9536,4.704403e+09,Leave,TRACER_Loop,0,1,None


In [15]:
# List of dictionaries (stored row-wise)
events = df.to_dict(orient="records")

## Experiment 1: Summing the "Timestamp (ns)" column

In [22]:
%%timeit
df["Timestamp (ns)"].sum()

49.5 µs ± 787 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [81]:
%%timeit
sum(df["Timestamp (ns)"].tolist())

165 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [85]:
%%timeit
sum([event["Timestamp (ns)"] for _, event in df.iterrows()])

427 ms ± 7.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
%%timeit
total = 0
for idx, event in df.iterrows():
    total += event["Timestamp (ns)"]

439 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%%timeit
sum(evt["Timestamp (ns)"] for evt in events)

473 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [79]:
%%timeit
sum(map(itemgetter("Timestamp (ns)"), events))

434 µs ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


What results do we see here?

If we sum a column using Pandas (column-oriented), we get ~50 us.

If we sum a column using Python list of dicts (row-oriented), we get ~434 us (9.88x slower).


However, if we do this operation without vectorization (using row-by-row iteration), using Pandas we get ~427 ms.

Using Python list of dicts, this is still ~434 us, which is (983x faster).

In this case, it is **worth moving to a row-oriented format.**

## What if we used DuckDB?

DuckDB is also column-oriented, so it's a bad idea to add one row at a time. However, DuckDB lets us do so since it uses SQL.

From their doc:

> INSERT statements are the standard way of loading data into a relational database. When using INSERT statements, the values are supplied row-by-row. While simple, there is significant overhead involved in parsing and processing individual INSERT statements. This makes lots of individual row-by-row insertions very inefficient for bulk insertion.

> As a rule-of-thumb, avoid using lots of individual row-by-row INSERT statements when inserting more than a few rows (i.e., avoid using INSERT > statements as part of a loop). When bulk inserting data, try to maximize the amount of data that is inserted per statement.

So, a more efficient way will be doing so in batches, instead of individual event insertions. Thus, we'll need to keep some sort of buffer in Python, and then flush this buffer by inserting into a DuckDB table..



In [61]:
conn = duckdb.connect(database=':memory:')

conn.execute('''
    CREATE TABLE IF NOT EXISTS my_table (
        "Timestamp (ns)" double,
        "Event Type" varchar,
        "Name" varchar,
        "Thread" int32,
        "Process" varchar,
        "Attributes" varchar,
    )
''')
for event in events:
    timestamp = event["Timestamp (ns)"]
    event_type = event["Event Type"]
    name = event["Name"]
    thread = event["Thread"]
    process = event["Process"]
    attributes = json.dumps(event["Attributes"])

    # Use f-string to dynamically insert values with correct syntax
    conn.execute(f"INSERT INTO my_table VALUES ({timestamp}, '{event_type}', '{name}', {thread}, {process}, '{attributes}')")

In [57]:
%%timeit
conn.execute('SELECT SUM("Timestamp (ns)") FROM my_table').fetchall()

288 µs ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## What if we use Polars?

Since polars is column-oriented, it's not recommend to add one row at a time. Instead, we can build a Python list of dicts and then convert that to a Polars DataFrame.

In practice, this will duplicate memory, so we should build a small Polars DF at a time, freeing up that memory from the Python list. 

So effectively, this is the same as the DuckDB approach -- build a buffer in Python, push it on to the DuckDB/Polars table, and flush the buffer.

In [58]:
podf = polars.DataFrame(events)
podf

Timestamp (ns),Event Type,Name,Thread,Process,Attributes
f64,str,str,i64,i64,struct[4]
0.477292,"""Instant""","""ProgramBegin""",0,14,"{""/g/g92/bhowmik1/sw4lite/sw4lite/optimize_c_quartz/sw4lite"",[""tests/trace-collection-input/gaussian-32.in""],null,null}"
148.437925,"""Instant""","""ProgramBegin""",0,15,"{""/g/g92/bhowmik1/sw4lite/sw4lite/optimize_c_quartz/sw4lite"",[""tests/trace-collection-input/gaussian-32.in""],null,null}"
1740.207954,"""Instant""","""ProgramBegin""",0,12,"{""/g/g92/bhowmik1/sw4lite/sw4lite/optimize_c_quartz/sw4lite"",[""tests/trace-collection-input/gaussian-32.in""],null,null}"
20025.75566,"""Instant""","""ProgramBegin""",0,13,"{""/g/g92/bhowmik1/sw4lite/sw4lite/optimize_c_quartz/sw4lite"",[""tests/trace-collection-input/gaussian-32.in""],null,null}"
20142.692289,"""Enter""","""MPI_Init""",0,14,"{null,null,""Region 36"",null}"
20377.520131,"""Enter""","""MPI_Init""",0,12,"{null,null,""Region 36"",null}"
20534.072026,"""Enter""","""MPI_Init""",0,15,"{null,null,""Region 36"",null}"
35336.817362,"""Enter""","""MPI_Init""",0,13,"{null,null,""Region 36"",null}"
80158.388554,"""Instant""","""ProgramBegin""",0,10,"{""/g/g92/bhowmik1/sw4lite/sw4lite/optimize_c_quartz/sw4lite"",[""tests/trace-collection-input/gaussian-32.in""],null,null}"


In [91]:
%%timeit
podf["Timestamp (ns)"].sum()

3.38 µs ± 53.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [65]:
%%timeit
total = 0
for row in podf.iter_rows():
    total += row[0]

7.75 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
%%timeit
total = 0
for row in podf.select(['Timestamp (ns)']).iter_rows():
    total += row[0]

1.09 ms ± 8.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [86]:
%%timeit
sum(map(itemgetter("Timestamp (ns)"), events))

448 µs ± 50.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


What results do we see here?

If we sum a column using Polars (column-oriented), we get ~4 us.

If we sum a column using Python list of dicts (row-oriented), we get ~448 us (112x slower).

However, if we do this operation without vectorization (using row-by-row iteration), using Polars we get either ~7.75ms or ~1.09 ms, depending on how many columns we're using.

Using Python list of dicts, this is still ~448 us, which is (17.3.x faster) or (2.43x faster).

In this case, it is **not worth moving to row-wise format**.

# Is there a better way to calculate exclusive time?

Right now, we're doing a $O(n^2)$ operation. Can we do better?